<a href="https://colab.research.google.com/github/almir-martins/migrando-pacotes-excel/blob/main/Simulando_Pacotes_Correios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulação descontos e contrapartida na troca de pacotes de serviços

### Simulação de migração dos pacotes de serviços antigos para os  pacotes da Nova Política Comercial usando a planilha fornecida em Excel. 

### Usando a biblioteca <a href="https://github.com/burnash/gspread"><code>gspread</code></a> de código aberto para interagir com o Planilhas Google.



In [2]:
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


### Importar a biblioteca
### Autenticação no Google Planilhas.

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# Pega os cabeçalhos das colunas da planilha SEDEX na versão do cliente

In [4]:
def pega_cabecalhos_sedex(arquivo):    
  # Seleciona a aba
  aba = 'Almir simulação'

  # Seleciona a planilha
  planilha = arquivo.worksheet(aba)

  # Pega os cabeçalhos
  cabecalhos = planilha.get('A3:M3')

  # Converte para dataframe
  df_cabecalhos = pd.DataFrame.from_records(cabecalhos)

  return list(df_cabecalhos.loc[0])

# Método para formatar tabela de SEDEX

In [5]:
def formata_tabela_sedex(nome_planilha, arquivo): 
  # Seleciona Planilha do pacote do cliente
  planilha = arquivo.worksheet(nome_planilha)

  # Tabela de preços do cliente (PAC)
  tabela_sedex = planilha.get('A5:U18')  

  # Converte para dataframe
  df_tabela_sedex = pd.DataFrame.from_records(tabela_sedex)

  # Apaga a primeira linha
  df_tabela_sedex = df_tabela_sedex.drop([0,0])

  # Apaga as colunas desnecessárias
  df_tabela_sedex.drop(df_tabela_sedex.columns[[2,3,4,6,7,8,14,20]], axis=1, inplace=True)

  # Pega os cabeçalhos definitivos
  df_tabela_sedex.columns = pega_cabecalhos_sedex(arquivo)

  return df_tabela_sedex


# Pega os cabeçalhos das colunas da planilha PAC na versão do cliente

In [6]:
def pega_cabecalhos_pac(arquivo):    
  # Seleciona a aba
  aba = 'Almir simulação'

  # Seleciona a planilha
  planilha = arquivo.worksheet(aba)

  # Pega os cabeçalhos
  cabecalhos = planilha.get('A21:L21')

  # Converte para dataframe
  df_cabecalhos = pd.DataFrame.from_records(cabecalhos)

  return list(df_cabecalhos.loc[0])

# Método para formatar tabela de PAC

In [7]:
def formata_tabela_pac(nome_planilha, arquivo): 
  # Seleciona Planilha do pacote do cliente
  planilha = arquivo.worksheet(nome_planilha)

  # Tabela de preços do cliente (PAC)
  tabela_pac = planilha.get('A24:Q36')  

  # Converte para dataframe
  df_tabela_pac = pd.DataFrame.from_records(tabela_pac)

  # Apaga a primeira linha
  df_tabela_pac = df_tabela_pac.drop([0,0])

  # Apaga as colunas desnecessárias
  df_tabela_pac.drop(df_tabela_pac.columns[[2,3,4,10,16]], axis=1, inplace=True)

  # Pega os cabeçalhos definitivos
  df_tabela_pac.columns = pega_cabecalhos_pac(arquivo)

  return df_tabela_pac


# Fábrica de pacotes

## Constrói as tabelas de SEDEX e PAC do pacote requisitado

In [8]:
def gera_pacote(pacote):
  # Abre o arquivo
  arquivo = gc.open('Simulador de migração - Pacotes Python')

  # constrói a tabela SEDEX
  tabela_SEDEX = formata_tabela_sedex(pacote, arquivo)

  # constrói a tabela PAC
  tabela_PAC = formata_tabela_pac(pacote, arquivo)  

  return tabela_SEDEX, tabela_PAC

# Exclui a primeira coluna (não numérica) e formata os valores como float

In [9]:
# Formata os valores para float e exclui colunas de texto
def to_float(pacote):
  pacote_interno = pacote.copy()
  pacote_interno = pacote_interno.drop(columns=['Peso(gr)'])

  for coluna in pacote_interno.columns:
    pacote_interno[coluna] = pacote_interno[coluna].str.replace(',', '.')
    pacote_interno[coluna] = pacote_interno[coluna].astype(float)

  return pacote_interno

# Calculo o desconto médio entre dois pacotes

In [15]:
def calcula_desconto_medio(pacote_antigo, pacote_novo):
  pacote_old = pacote_antigo.copy()
  pacote_new = pacote_novo.copy()

  # Formata os pacotes para float
  pacote_old = to_float(pacote_old)
  pacote_new = to_float(pacote_new)


  # Planilha com os descontos/acrécimos em percentual
  planilha_descontos = ((pacote_new - pacote_old) / pacote_old)
  # Média geral dos percentuais
  desconto = planilha_descontos.stack().mean()

  return desconto * 100

# Executável

In [23]:
import pandas as pd
import numpy as np
from google.colab import files 

# Pacote de origem e pacote visado
pacote_cliente = 'ENC_2.6'
pacote_novo = 'OURO3'

# Gera as planilhas definitivas
pac_cliente, sedex_cliente = gera_pacote(pacote_cliente)
pac_pacote, sedex_pacote = gera_pacote(pacote_novo)

desconto_pac = calcula_desconto_medio(pac_cliente, pac_pacote)
desconto_sedex = calcula_desconto_medio(sedex_cliente, sedex_pacote)

# print(pac_cliente.to_string(index=False))
# print(pac_pacote.to_string(index=False))
# print(f'O desconto médio no PAC em relação ao pacote antigo é {desconto_pac}%')
# print(f'O desconto médio no SEDEX em relação ao pacote antigo é {desconto_sedex}%')

print(f'Pacote {pacote_novo}: Cota mínima de R$10.000,00 – Comparando com {pacote_cliente}')
print(f'Migrando você terá uma AUMENTO na média de {desconto_sedex:.2f}% no SEDEX e REDUÇÃO na média de {desconto_pac:.2f}% no PAC.')

Pacote OURO3: Cota mínima de R$10.000,00 – Comparando com ENC_2.6
Migrando você terá uma AUMENTO na média de -1.44% no SEDEX e REDUÇÃO na média de 1.66% no PAC.


In [62]:
# Converte para float
pac_cliente = to_float(pac_cliente)
sedex_cliente = to_float(sedex_cliente)
pac_pacote = to_float(pac_pacote)
sedex_pacote = to_float(sedex_pacote)

# Planilha com os descontos/acrécimos em percentual
planilha_descontos = ((sedex_pacote - sedex_cliente) / sedex_cliente)

print(planilha_descontos.head(13))

    SP (interior)  ...  Interior de: AC e RR
1       -0.014258  ...             -0.014415
2       -0.014519  ...             -0.014358
3       -0.013785  ...             -0.014302
4       -0.014416  ...             -0.014205
5       -0.014401  ...             -0.014328
6       -0.014316  ...             -0.014333
7       -0.013978  ...             -0.014271
8       -0.013983  ...             -0.014294
9       -0.014383  ...             -0.014261
10      -0.013981  ...             -0.014294
11      -0.014384  ...             -0.014297
12      -0.013812  ...             -0.015005

[12 rows x 11 columns]
